### Inner Join Operations

Core Idea:
1. left join didn't alleviate the problem with multiple NaN in data frame. Experiment with inner join
2. Remove entries which contain duplicated doc_id
3. Remove NaNs

In [20]:
from loguru import logger


from slt_positional_bias.dataset import generate_data_frame, generate_merged_data_frame

df_docs = generate_data_frame("data/interim/inputs/corpus.jsonl")
df_topics = generate_data_frame("data/interim/inputs/queries.jsonl")
df_qrels = generate_data_frame("data/interim/inputs/qrels.rag24.test-umbrela-all.txt")

df_qrels.columns = ["topic_id", "q0", "doc_id", "rel_scoring"]

df_topics = df_topics.rename(columns={"qid": "topic_id","query": "topic"})
df_docs = df_docs.rename(columns={"docno": "doc_id", "text": "doc"})

logger.info(f"df_qrels column 'topic_id' is unique: {df_qrels["topic_id"].is_unique}")
logger.info(f"df_topics column 'topic_id' is unique: {df_topics["topic_id"].is_unique}")
logger.info(f"df_qrels column 'doc_id' is unique: {df_qrels["doc_id"].is_unique}")
logger.info(f"df_docs column 'doc_id' is unique: {df_docs["doc_id"].is_unique}")
logger.info("--------------------------------------")

df_merged = (
    df_qrels
    .loc[:, ["topic_id", "doc_id", "rel_scoring"]]
    .merge(
        df_topics.loc[:, ["topic_id", "topic"]],
        on="topic_id",
        how="inner"
    )
    .merge(
        df_docs.loc[:, ["doc_id", "doc"]],
        on="doc_id",
        how="inner"
    )
)

# df_merged
num_topics = df_merged["topic_id"].nunique()
num_topics

2025-07-27 19:39:38.851 | INFO     | slt_positional_bias.dataset:generate_data_frame:128 - file path exists
2025-07-27 19:39:44.263 | INFO     | slt_positional_bias.dataset:generate_data_frame:128 - file path exists
2025-07-27 19:39:44.270 | INFO     | slt_positional_bias.dataset:generate_data_frame:128 - file path exists
2025-07-27 19:39:44.408 | INFO     | __main__:<module>:15 - df_qrels column 'topic_id' is unique: False
2025-07-27 19:39:44.408 | INFO     | __main__:<module>:16 - df_topics column 'topic_id' is unique: True
2025-07-27 19:39:44.426 | INFO     | __main__:<module>:17 - df_qrels column 'doc_id' is unique: False
2025-07-27 19:39:44.455 | INFO     | __main__:<module>:18 - df_docs column 'doc_id' is unique: True
2025-07-27 19:39:44.455 | INFO     | __main__:<module>:19 - --------------------------------------


89

In [10]:
df_qrels

,topic_id,q0,doc_id,rel_scoring
0,2024-145979,Q0,msmarco_v2.1_doc_01_523681915#0_449763684,2
1,2024-145979,Q0,msmarco_v2.1_doc_13_1647729865#0_3617397938,3
2,2024-145979,Q0,msmarco_v2.1_doc_25_1148700328#0_2179512504,2
3,2024-145979,Q0,msmarco_v2.1_doc_25_1148700328#1_2179514079,2
4,2024-145979,Q0,msmarco_v2.1_doc_44_1043805224#2_2182641075,2
...,...,...,...,...
108473,2024-21669,Q0,msmarco_v2.1_doc_08_298503251#1_568833050,0
108474,2024-21669,Q0,msmarco_v2.1_doc_27_877180220#2_1765802089,1
108475,2024-21669,Q0,msmarco_v2.1_doc_01_1752638978#7_2584141250,1
108476,2024-21669,Q0,msmarco_v2.1_doc_09_914336366#11_1554208777,0


In [11]:
df_docs

,doc_id,doc,original_document
0,msmarco_v2.1_doc_52_1400719578#0_2842698387,"Taylor Swift's ""Invisible String"" Lyrics Meani...",{'doc_id': 'msmarco_v2.1_doc_52_1400719578#0_2...
1,msmarco_v2.1_doc_05_1495231781#3_2850145650,History of Ancient Rome for Kids: The Roman Em...,{'doc_id': 'msmarco_v2.1_doc_05_1495231781#3_2...
2,msmarco_v2.1_doc_12_1016316941#1_1992316811,Apiphobia and Spheksophobia - Fear Of Wasps An...,{'doc_id': 'msmarco_v2.1_doc_12_1016316941#1_1...
3,msmarco_v2.1_doc_34_1545449353#0_3251041439,Reies Tijerina | American activist | Britannic...,{'doc_id': 'msmarco_v2.1_doc_34_1545449353#0_3...
4,msmarco_v2.1_doc_23_1497031463#10_3313663249,"Salome: Was the ""dancing"" daughter of Herodias...",{'doc_id': 'msmarco_v2.1_doc_23_1497031463#10_...
...,...,...,...
116689,msmarco_v2.1_doc_56_94836053#11_205606001,TSHA | DeWitt's Colony DeWitt's Colony\nSuppor...,{'doc_id': 'msmarco_v2.1_doc_56_94836053#11_20...
116690,msmarco_v2.1_doc_44_596645538#4_1407144514,SAFE Investment Company (China) Definition SAF...,{'doc_id': 'msmarco_v2.1_doc_44_596645538#4_14...
116691,msmarco_v2.1_doc_49_1086472702#22_2229413969,Money is Everything - Physician on FIRE Money ...,{'doc_id': 'msmarco_v2.1_doc_49_1086472702#22_...
116692,msmarco_v2.1_doc_34_1540555152#9_3240635353,"Maximilien Robespierre | Biography, French Rev...",{'doc_id': 'msmarco_v2.1_doc_34_1540555152#9_3...


In [12]:
df_topics

,topic_id,topic,original_query
0,2024-145979,what is vicarious trauma and how can it be cop...,"{'query_id': '2024-145979', 'text': 'what is v..."
1,2024-216592,why disability insurance is a smart investment,"{'query_id': '2024-216592', 'text': 'why disab..."
2,2024-32912,how bad did the vietnam war devastate the econ...,"{'query_id': '2024-32912', 'text': 'how bad di..."
3,2024-153051,what target stors's policies for shoplifting,"{'query_id': '2024-153051', 'text': 'what targ..."
4,2024-128784,what can a career in civil engineering be like?,"{'query_id': '2024-128784', 'text': 'what can ..."
...,...,...,...
84,2024-219563,why free things have less percieved value,"{'query_id': '2024-219563', 'text': 'why free ..."
85,2024-121840,should teachers notify parents about state tes...,"{'query_id': '2024-121840', 'text': 'should te..."
86,2024-58819,how many latino have been misinformed about ed...,"{'query_id': '2024-58819', 'text': 'how many l..."
87,2024-42014,how does religion show in public school,"{'query_id': '2024-42014', 'text': 'how does r..."


In [13]:
logger.info(f"df_merged column 'doc_id' is unique: {df_merged["doc_id"].is_unique}")

2025-07-27 17:29:46.866 | INFO     | __main__:<module>:1 - df_merged column 'doc_id' is unique: False


In [14]:
dupes = df_merged[df_merged["doc_id"].duplicated(keep=False)]
dupes

,topic_id,doc_id,rel_scoring,topic,doc
638,2024-32912,msmarco_v2.1_doc_21_1227054210#2_2626884929,1,how bad did the vietnam war devastate the econ...,"Considering the Losses, Does it Matter Who Won..."
1513,2024-42497,msmarco_v2.1_doc_34_1636826970#6_3436858544,3,how does the informal recycling sector add to ...,Electronic waste | Britannica Electronic waste...
1514,2024-42497,msmarco_v2.1_doc_08_1276221077#1_2305275296,3,how does the informal recycling sector add to ...,The Human and Environmental Effects of E-Waste...
1516,2024-42497,msmarco_v2.1_doc_17_458289946#6_496645752,1,how does the informal recycling sector add to ...,Electronic waste in China - Wikipedia Electron...
1517,2024-42497,msmarco_v2.1_doc_08_1276221077#0_2305273239,3,how does the informal recycling sector add to ...,The Human and Environmental Effects of E-Waste...
...,...,...,...,...,...
29143,2024-127266,msmarco_v2.1_doc_34_920686972#4_1884834759,1,what are some key challenges related to the re...,How Does Waste Impact The Environment? How Doe...
29147,2024-127266,msmarco_v2.1_doc_17_458289946#6_496645752,2,what are some key challenges related to the re...,Electronic waste in China - Wikipedia Electron...
29148,2024-127266,msmarco_v2.1_doc_25_186084585#1_358397316,1,what are some key challenges related to the re...,How to Start E-waste Recycling Business in Ind...
29149,2024-127266,msmarco_v2.1_doc_51_880750803#8_1852939826,1,what are some key challenges related to the re...,Electronic Recycling: How to Recycle Your Busi...


In [15]:
dupe_638 = df_merged[df_merged["doc_id"] == "msmarco_v2.1_doc_21_1227054210#2_2626884929"]
dupe_1513 = df_merged[df_merged["doc_id"] == "msmarco_v2.1_doc_34_1636826970#6_3436858544"]
dupe_638


,topic_id,doc_id,rel_scoring,topic,doc
638,2024-32912,msmarco_v2.1_doc_21_1227054210#2_2626884929,1,how bad did the vietnam war devastate the econ...,"Considering the Losses, Does it Matter Who Won..."
20856,2024-213978,msmarco_v2.1_doc_21_1227054210#2_2626884929,0,why are french farmers protesting,"Considering the Losses, Does it Matter Who Won..."


In [16]:
dupe_1513

,topic_id,doc_id,rel_scoring,topic,doc
1513,2024-42497,msmarco_v2.1_doc_34_1636826970#6_3436858544,3,how does the informal recycling sector add to ...,Electronic waste | Britannica Electronic waste...
28964,2024-127266,msmarco_v2.1_doc_34_1636826970#6_3436858544,3,what are some key challenges related to the re...,Electronic waste | Britannica Electronic waste...


In [17]:
df_clean = df_merged[df_merged["doc_id"].duplicated(keep=False) == False]
logger.info(f"df_clean column 'doc_id' is unique: {df_clean["doc_id"].is_unique}")
df_clean


2025-07-27 17:29:47.780 | INFO     | __main__:<module>:2 - df_clean column 'doc_id' is unique: True


,topic_id,doc_id,rel_scoring,topic,doc
0,2024-145979,msmarco_v2.1_doc_01_523681915#0_449763684,2,what is vicarious trauma and how can it be cop...,Vicarious traumatization - Wikipedia Vicarious...
1,2024-145979,msmarco_v2.1_doc_13_1647729865#0_3617397938,3,what is vicarious trauma and how can it be cop...,Vicarious Trauma: What Causes it and Ways to H...
2,2024-145979,msmarco_v2.1_doc_25_1148700328#0_2179512504,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
3,2024-145979,msmarco_v2.1_doc_25_1148700328#1_2179514079,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
4,2024-145979,msmarco_v2.1_doc_44_1043805224#2_2182641075,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? - Jefferson Center -...
...,...,...,...,...,...
29158,2024-127266,msmarco_v2.1_doc_58_1230815815#5_1795053441,2,what are some key challenges related to the re...,Highest E-Waste Generating Nations In The Worl...
29159,2024-127266,msmarco_v2.1_doc_34_470264225#3_1018882699,0,what are some key challenges related to the re...,The Pros & Cons Of Landfills (Benefits & Disad...
29160,2024-127266,msmarco_v2.1_doc_57_781229369#6_1221950031,1,what are some key challenges related to the re...,The Patron Saint of Electronics? Do you Know t...
29161,2024-127266,msmarco_v2.1_doc_57_781235684#6_1221962946,1,what are some key challenges related to the re...,Destroy VHS Tapes \nDestroy VHS Tapes\nDaliah ...


In [18]:
df_clean = df_clean.dropna()
df_clean

,topic_id,doc_id,rel_scoring,topic,doc
0,2024-145979,msmarco_v2.1_doc_01_523681915#0_449763684,2,what is vicarious trauma and how can it be cop...,Vicarious traumatization - Wikipedia Vicarious...
1,2024-145979,msmarco_v2.1_doc_13_1647729865#0_3617397938,3,what is vicarious trauma and how can it be cop...,Vicarious Trauma: What Causes it and Ways to H...
2,2024-145979,msmarco_v2.1_doc_25_1148700328#0_2179512504,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
3,2024-145979,msmarco_v2.1_doc_25_1148700328#1_2179514079,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? | What is Vicarious ...
4,2024-145979,msmarco_v2.1_doc_44_1043805224#2_2182641075,2,what is vicarious trauma and how can it be cop...,What is Vicarious Trauma? - Jefferson Center -...
...,...,...,...,...,...
29158,2024-127266,msmarco_v2.1_doc_58_1230815815#5_1795053441,2,what are some key challenges related to the re...,Highest E-Waste Generating Nations In The Worl...
29159,2024-127266,msmarco_v2.1_doc_34_470264225#3_1018882699,0,what are some key challenges related to the re...,The Pros & Cons Of Landfills (Benefits & Disad...
29160,2024-127266,msmarco_v2.1_doc_57_781229369#6_1221950031,1,what are some key challenges related to the re...,The Patron Saint of Electronics? Do you Know t...
29161,2024-127266,msmarco_v2.1_doc_57_781235684#6_1221962946,1,what are some key challenges related to the re...,Destroy VHS Tapes \nDestroy VHS Tapes\nDaliah ...
